# **Sentiment Analysis of Product Reviews - ETL**

### **Objectives**

In this notebook we will load the raw dataset 'Reviews.csv' from Kaggle  (https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews). We will perform the ETL(Extract, Transform and Load) process on the dataset by cleaning it, ensuring that all required variables are present and correctly formatted, and then saving the resulting 'cleaned' dataset for subsequent analyses.

### **Inputs**

Raw dataset from Kaggle 'Reviews.csv'

### **Outputs**

Cleaned dataset saved as a new CSV file.

### **Prerequisites**

* Python 3.12.8 is installed
* Required Python libraries from requirement.txt and their dependencies must be installed
* Optional to set up Python virtual environment



---

# Extract: Import data and libraries

### Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [2]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\admin\\Documents\\vscode-projects\\ci-hackathon-sentiment-analysis-of-product-reviews\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\admin\\Documents\\vscode-projects\\ci-hackathon-sentiment-analysis-of-product-reviews'

### Import libraries

Load required packages

In [5]:
import pandas as pd
import numpy as np

### Load data

In [6]:
df_raw = pd.read_csv('./data/Reviews.csv')
print(df_raw.shape)
df_raw.head()

(568454, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


---

# Transform: Clean data

Collect information about the data

In [7]:
# Check info
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568428 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [8]:
# Check columns
df_raw.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [9]:
# Check data types
df_raw.dtypes

Id                         int64
ProductId                 object
UserId                    object
ProfileName               object
HelpfulnessNumerator       int64
HelpfulnessDenominator     int64
Score                      int64
Time                       int64
Summary                   object
Text                      object
dtype: object

The presence of missing values can be checked using .isnull().sum(), which counts the number of null entries in each column. If all columns return 0, it means there are no missing values

In [10]:
df_raw.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               26
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

There are 0 missing values in critical columns such as  Text and Score. This makes it so there is no need to drop any entries. There are 26 missing ProfileName values which can be filled by 'Anonymous'.

In [11]:
df_raw['ProfileName'] = df_raw['ProfileName'].fillna('Anonymous')

There are 27 missing values in Summary. This isn't a critical column either so we can fill with empty string so its cleaner.

In [12]:
df_raw['Summary'] = df_raw['Summary'].fillna('')

We can check again for missing values.

In [13]:
df_raw.isnull().sum()

Id                        0
ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64

Now, we will check for duplicates. Duplicates inflates results and bias the model.

In [14]:
print(f"Total reviews: {len(df_raw):,}")
print(f"Duplicate rows (all columns): {df_raw.duplicated().sum():,}")
print(f"Duplicate reviews (same user, product, time, text): {df_raw.duplicated(subset=['UserId', 'ProductId', 'Time', 'Text']).sum():,}")
print(f"Duplicate text only: {df_raw.duplicated(subset=['Text']).sum():,}")

Total reviews: 568,454
Duplicate rows (all columns): 0
Duplicate reviews (same user, product, time, text): 1,205
Duplicate text only: 174,875


The 1,205 duplicate reviews mean that the same person reviewed the same product at the same time with identical text. This is likely accidental duplicate submissions. We can remove these as they are genuine duplicates.

In [15]:
df_raw = df_raw.drop_duplicates(subset=['UserId', 'ProductId', 'Time', 'Text'], keep='first')
print(f"Removed {1205} duplicate reviews")

Removed 1205 duplicate reviews


We can check agsin for duplicates.

In [16]:
print(f"Total reviews: {len(df_raw):,}")
print(f"Duplicate reviews (same user, product, time, text): {df_raw.duplicated(subset=['UserId', 'ProductId', 'Time', 'Text']).sum():,}")
print(f"Duplicate text only: {df_raw.duplicated(subset=['Text']).sum():,}")

Total reviews: 567,249
Duplicate reviews (same user, product, time, text): 0
Duplicate text only: 173,670


In [17]:
# Quick investigation - see what those duplicates are
text_counts = df_raw['Text'].value_counts()
duplicated = text_counts[text_counts > 1]

print(f"Total reviews: {len(df_raw):,}")
print(f"Texts appearing multiple times: {len(duplicated):,}\n")

print("Top 30 most duplicated reviews:")
print("="*70)
for i, (text, count) in enumerate(duplicated.head(30).items(), 1):
    words = len(text.split())
    preview = text[:80] + "..." if len(text) > 80 else text
    print(f"{i:2d}. {count:>6,}x ({words:>2} words): '{preview}'")

Total reviews: 567,249
Texts appearing multiple times: 58,004

Top 30 most duplicated reviews:
 1.    103x (506 words): 'Diamond Almonds<br />Almonds are a good source of magnesium. One ounce contain a...'
 2.     38x (864 words): 'This review will make me sound really stupid, but whatever. I don't really care ...'
 3.     38x (108 words): 'Stash Chamomile Herbal Tea is tea bags with dried, crushed chamomile flowers.<br...'
 4.     28x (242 words): 'I'm addicted to salty and tangy flavors, so when I opened my first bag of Sea Sa...'
 5.     26x (142 words): 'I'm not a tea drinker, so I'll start with that right off the bat.  However, I wa...'
 6.     26x (30 words): 'I'd continue to buy but I'm moving over to more home made finger foods and away ...'
 7.     25x (131 words): 'I drank this tea at least 4 days a week during the last 6 weeks of my pregnancy....'
 8.     25x (32 words): 'I bought this for my wife who is pregnant, and she drinks it everyday and likes ...'
 9.     25x (64 wor

Generic short reviews ("Great!", "Love it!"). These are long detailed reviews appearing several times which means there are clear signs of fake/bot reviews.

In [18]:
# Remove any review text appearing 10+ times

print(f"Before: {len(df_raw):,} reviews")

# Get text frequency
text_counts = df_raw['Text'].value_counts()

# Find suspicious texts (appearing 10+ times)
suspicious = text_counts[text_counts >= 10].index

# Remove all but first occurrence
for text in suspicious:
    duplicates = df_raw[df_raw['Text'] == text].index[1:]  # Get all except first
    df_raw = df_raw.drop(duplicates)

print(f"After: {len(df_raw):,} reviews")
print(f"Removed: {567249 - len(df_raw):,} reviews")

Before: 567,249 reviews
After: 533,650 reviews
Removed: 33,599 reviews


In [19]:
# Final check for duplicates
text_counts = df_raw['Text'].value_counts()
print(f"Reviews: {len(df_raw):,}")
print(f"Max duplicates: {text_counts.max()}")
print(f"Texts with 10+ occurrences: {(text_counts >= 10).sum()}")

Reviews: 533,650
Max duplicates: 9
Texts with 10+ occurrences: 0


In [ ]:
# SIMPLE HTML REMOVAL
import re

def simple_clean_html(text):
    if pd.isna(text):
        return ""
    text = str(text)
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Replace common HTML entities
    text = text.replace('&amp;', '&')
    text = text.replace('&lt;', '<')
    text = text.replace('&gt;', '>')
    text = text.replace('&quot;', '"')
    text = text.replace('&#39;', "'")
    text = text.replace('&nbsp;', ' ')
    text = text.replace('<br>', ' ')
    text = text.replace('<br/>', ' ')
    return text

print("Cleaning HTML with simple method...")
df_raw['Text'] = df_raw['Text'].apply(simple_clean_html)
df_raw['Summary'] = df_raw['Summary'].apply(simple_clean_html)
print("✓ HTML cleaned")

Cleaning HTML with simple method...
✓ HTML cleaned


HTML cleaning removes code/formatting tags that shouldn't be in review text.

In [21]:
# Fix invalid helpfulness ratios
print("Fixing helpfulness ratios...")

# Find invalid ratios (numerator > denominator)
invalid = df_raw['HelpfulnessNumerator'] > df_raw['HelpfulnessDenominator']

print(f"Invalid ratios found: {invalid.sum():,}")

if invalid.sum() > 0:
    # Fix by capping numerator at denominator
    df_raw.loc[invalid, 'HelpfulnessNumerator'] = df_raw.loc[invalid, 'HelpfulnessDenominator']
    print(f"✓ Fixed {invalid.sum():,} invalid ratios")
else:
    print("✓ No invalid ratios - data is clean!")

print(f"\nFinal dataset: {len(df_raw):,} reviews")

Fixing helpfulness ratios...
Invalid ratios found: 2
✓ Fixed 2 invalid ratios

Final dataset: 533,650 reviews


At this stage, after handling missing values, identifying and removing duplicates, and performing HTML content cleaning, we decided it was best to sample the dataset before proceeding with further analysis. This approach allows us to validate the effectiveness of our cleaning steps, ensure data consistency, and efficiently assess the overall data quality before applying more complex transformations or analytical processes.

In [22]:
# Stratified sample
print("Creating stratified sample of 20,000...")

df_sample = df_raw.groupby('Score', group_keys=False).apply(
    lambda x: x.sample(min(len(x), int(20000 * len(x) / len(df_raw))), random_state=42)
)

# If slightly under 20k due to rounding, top up
if len(df_sample) < 20000:
    remaining = df_raw[~df_raw.index.isin(df_sample.index)]
    extra = remaining.sample(n=20000 - len(df_sample), random_state=42)
    df_sample = pd.concat([df_sample, extra])

print(f"✅ Sample created: {len(df_sample):,} reviews")

Creating stratified sample of 20,000...
✅ Sample created: 20,000 reviews


Stratified sampling helps you get a more representative sample of the entire dataset, especially when certain subgroups are small or very different from others. This stratifies sampling is based on Score as it maintains the same proportion of 1-5 star ratings in the sample as in the full dataset.
This was the chosen category as the sample:
* Keeps real-world distribution
* Not biased toward any rating
* Analysis results will match full dataset

In [23]:
# Check sample shape
df_sample.shape

(20000, 10)

In [24]:
# Convert Unix timestamp to datetime
df_sample['Date'] = pd.to_datetime(df_sample['Time'], unit='s')

print("✅ Converted Time to Date")
print(f"\nDate range: {df_sample['Date'].min()} to {df_sample['Date'].max()}")
print(f"\nFirst few dates:")
print(df_sample[['Time', 'Date']].head())

✅ Converted Time to Date

Date range: 2000-01-03 00:00:00 to 2012-10-26 00:00:00

First few dates:
              Time       Date
406805  1342396800 2012-07-16
75917   1310947200 2011-07-18
400894  1274486400 2010-05-22
480704  1350086400 2012-10-13
103599  1307404800 2011-06-07


The code above converts the Unix timestamps in the Time column into human-readable datetime values and stores them in a new column called Date.

In [26]:
# Add Sentiment column
df_sample['Sentiment'] = df_sample['Score'].apply(lambda x: 'Negative' if x <= 2 else ('Neutral' if x == 3 else 'Positive'))

print("✅ Sentiment column added!")
print(df_sample['Sentiment'].value_counts())

✅ Sentiment column added!
Sentiment
Positive    15618
Negative     2886
Neutral      1496
Name: count, dtype: int64


This code above creates a new column called Sentiment based on the numerical values in the Score column.
Each score is mapped to a sentiment category using a simple rule:

* Scores 1 or 2 = “Negative”

* Score 3 = “Neutral”

* Scores 4 or 5 = “Positive”

This transformation converts numerical ratings into qualitative sentiment labels, making the data easier to interpret and analyse.

---

# Load: Save data

We now have a cleaned data to be saved as a new csv file. We will save the full cleaned data and the sample of the cleaned data as separate files.

In [28]:
# Save full cleaned data
df_raw.to_csv('./data/Reviews_Cleaned.csv', index=False)

# Save sample cleaned data
df_sample.to_csv('./data/Reviews_Sample_Cleaned.csv', index=False)

---

# Summary

* Changed work directory
* Imported required libraries
* Extracted data
* Collected insights regarding the data
* Cleaned data(missing values, duplicate values, HTML cleaning)
* Sampled the dataset for further analysis
* Converted Unix timestamps to readable date values
* Created a new sentiment column based on review scores
* Exported the cleaned data and processed data to new CSV files